In [ ]:
import pandas as pd
import re
import warnings
warnings.filterwarnings('ignore')
from ipl_parser import IPLParser
pd.set_option('display.max_rows', 200)
from player_details import player_details
from number_mapping import number_mapping
from name_mapping import name_mapping

In [ ]:
Input_Filepath = 'D:\\Datasets\\IPL_2024\\KKR VS SRH'
Output_FilePath = 'D:\\Datasets\\IPL_2024\\Data Files'
stadium = 'Eden Gardens, Kolkata'

parser = IPLParser(Input_Filepath, Output_FilePath,stadium,3)

In [ ]:
mydata = parser.getMatchData()

In [ ]:
mydata.shape

In [ ]:
def get_bowler_batter_outcom(x):
    return re.search(r'(\w+\s?\w*)\s+to', x).group().replace('to','').strip() , re.search(r'to\s+([\w\s]+)', x).group(1) , re.search(r'<span>(.*?)<\/span>', x).group(1)\

def get_ball_details(x):
    return re.search(r'<span>(.*?)<\/span>', str(x)).group(1)

In [ ]:
mydata[['Bowler','Batter','Outcome']] = mydata['batter_bowler'].apply(lambda x : pd.Series(get_bowler_batter_outcom(x)))

mydata['Runs'] = mydata['ball_details'].apply(lambda x : get_ball_details(x))

In [ ]:
mydata['Batter'].unique()

In [ ]:
mydata['Bowler'].unique()

In [ ]:
mydata[['Batter','Bowler']] = mydata[['Batter','Bowler']].replace(name_mapping)

In [ ]:
mydata[['Bowler','Batter']]

In [ ]:
mydata['Batting Style'] = ''
mydata['Bowling Style'] = ''
mydata['Origin'] = ''

for index, row in mydata.iterrows():
    batter_name = row['Batter']
    bowler_name = row['Bowler']
    
    if batter_name in player_details:
        mydata.loc[index, 'Batting Style'] = player_details[batter_name]['batting_side']
        mydata.loc[index, 'Origin'] = 'Indian' if 'Indian' in player_details[batter_name]['Origin'] else 'Overseas'
    if bowler_name in player_details:
        mydata.loc[index, 'Bowling Style'] = player_details[bowler_name]['bowling_side']
        mydata.loc[index, 'Origin'] = 'Indian' if 'Indian' in player_details[bowler_name]['Origin'] else 'Overseas'

In [ ]:
mydata.head(100)

In [ ]:
mydata['Runs'] = mydata['Runs'].replace(number_mapping)

mydata['Runs'] = mydata['Runs'].str.replace('[A-Za-z]', '')

mydata['Runs'] = mydata['Runs'].apply(lambda x : int(x))

mydata['ball'] = mydata['overs'].apply(lambda x : int(x.split('.')[1]))

mydata['over'] = mydata['overs'].apply(lambda x : int(x.split('.')[0]))

In [ ]:
mydata.drop(columns=['ball_details','batter_bowler','overs'],inplace=True)

mydata = mydata[['innings','over','ball','Batter','Bowler','Runs','Outcome','Batting Style','Bowling Style','comments','venue']]

mydata.sort_values(by=['innings','over','ball'],ignore_index = True , inplace=True)

In [ ]:
#mydata.to_csv('D:\\Datasets\\IPL_2024\\Data Files\\Clean_Data\\RCB_VS_CSK.csv',index=False)